In [1]:
#Bowel Movement Frequency
#Gut Microbiome Analysis
#James P. Johnson
#Credit to Christian Diener, PhD as noted
#v3-7-23

# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

# Get the library
from arivale_data_interface import *
import arivale_data_interface as adi

# Set display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.display import display

# Apply these settings so the library knows where the data is
adi.apply_isb_config()

# If you want plots to show up in the notebook you need to run this
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [2]:
#Obtain Bowel Movement Frequency (BMF) data:
bowel = adi.get_snapshot('assessments')
bowel = bowel.sort_values('days_in_program',ascending=True) # Sort df by days first
bowel = bowel.drop(columns = ['vendor','days_in_program','days_since_first_call','days_since_first_draw']) # Drop days and vendor, unneeded from this point
bowel = bowel.drop_duplicates(subset='public_client_id') # Keep only the first timepoint for each public_client_id
bowel = bowel.fillna(0) # Fill in empty BMF data with zero
bowel = bowel.set_index('public_client_id') # Arrange df by index of PCI
bowel = bowel[['assessment:digestion:bowel-movements:enum']]
bowel = bowel.rename(columns={"assessment:digestion:bowel-movements:enum":"bowel"}) # rename column to "bowel"
bowel = bowel[bowel['bowel'] != 0] #remove participants with no bowel data, reduce from 5764 -> 3955 individuals
bowel_list = ['(1) 2 or fewer times per week', '(2) 3-6 times per week', '(3) 1-3 times daily', '(4) 4+ times daily']
bowel_r = [1,2,3,4] #convert bowel responses to ordinal bowel movement frequency categories
bowel.bowel = bowel.bowel.replace(to_replace = bowel_list, value = bowel_r)
bowel = bowel.reset_index()
# 3955 individuals
bowel

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,bowel
0,01092042,3
1,01568974,3
2,01684342,2
3,01601230,3
4,01124029,3
...,...,...
3950,01395286,3
3951,01232614,3
3952,01181682,3
3953,01380110,3


In [5]:
#Import ASV count data:
asvs = pd.read_csv("/proj/arivale/microbiome/16S_processed/asvs.csv")
asvs.head()

,id,hash,count
0,22001612560016|GFM-1079-001,c2d3fc09212e226b3a2c3398a1af9436,1285
1,22001612560023|GFM-1079-001,c2d3fc09212e226b3a2c3398a1af9436,2431
2,22001612560062|GFM-1079-001,c2d3fc09212e226b3a2c3398a1af9436,4908
3,22001612560065|GFM-1079-001,c2d3fc09212e226b3a2c3398a1af9436,3944
4,22001612560067|GFM-1079-001,c2d3fc09212e226b3a2c3398a1af9436,5990


In [17]:
print("#samples:\t", asvs.id.nunique())
print("#sequences:\t", asvs.hash.nunique())

#samples:	 5232
#sequences:	 89022


In [7]:
samples = pd.read_csv("/proj/arivale/microbiome/16S_processed/metadata.csv")
merged = pd.merge(asvs, samples[["id", "public_client_id", "sex", "age", "days_in_program"]], on="id")

tax = pd.read_csv("/proj/arivale/microbiome/16S_processed/taxonomy.csv")
tax = tax.drop("sequence", axis=1).rename(columns={"id": "hash"})
merged = pd.merge(merged, tax, on="hash")

#Create taxa genus df
taxa = merged[['public_client_id','count','Kingdom','Phylum','Class','Order','Family','Genus'#,'Species'
              ]]

#Add prefixes to taxa names for consistency
taxa = taxa.assign(Name = 'taxa_' + taxa.Kingdom.astype(str) + '.' + \
  taxa.Phylum.astype(str) + '.' + taxa.Class.astype(str) + '.' + \
  taxa.Order.astype(str) + '.' + taxa.Family.astype(str) + '.' + \
  taxa.Genus.astype(str)# + '.' + taxa.Species.astype(str) \
                  )

taxa = taxa.sort_values(by = ['public_client_id'])

#create MultiIndex for pivot table
arrays = [taxa['public_client_id'],taxa['Name']]
index = pd.MultiIndex.from_arrays(arrays, names = ('public_client_id','Name'))

#pivot df into one with taxa as columns and PCI as index with counts as values
name_melt = pd.DataFrame(data = {'count':taxa['count'].to_numpy()}, index = index)

name_pivot = name_melt.pivot_table(index='public_client_id', columns='Name', values='count',aggfunc='sum', fill_value = 0)
name_pivot = name_pivot.dropna(how = 'any')
name_pivot

Name,taxa_Archaea.Euryarchaeota.Methanobacteria.Methanobacteriales.Methanobacteriaceae.Methanobrevibacter,taxa_Archaea.Euryarchaeota.Methanobacteria.Methanobacteriales.Methanobacteriaceae.Methanosphaera,taxa_Archaea.Euryarchaeota.Methanobacteria.Methanobacteriales.Methanobacteriaceae.nan,taxa_Archaea.Euryarchaeota.Thermoplasmata.Methanomassiliicoccales.Methanomassiliicoccaceae.Methanomassiliicoccus,taxa_Archaea.Euryarchaeota.Thermoplasmata.Methanomassiliicoccales.Methanomethylophilaceae.Candidatus_Methanogranum,taxa_Archaea.Euryarchaeota.Thermoplasmata.Methanomassiliicoccales.Methanomethylophilaceae.Candidatus_Methanomethylophilus,taxa_Archaea.Euryarchaeota.Thermoplasmata.Methanomassiliicoccales.Methanomethylophilaceae.nan,taxa_Bacteria.Actinobacteria.Acidimicrobiia.Microtrichales.nan.nan,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Actinobaculum,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Actinomyces,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Actinotignum,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Arcanobacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.F0332,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Mobiluncus,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Trueperella,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Varibaculum,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.nan,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Aeriscardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Alloscardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Bifidobacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Gardnerella,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Neoscardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Parascardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Pseudoscardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Scardovia,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.nan,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.Corynebacteriaceae.Corynebacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.Corynebacteriaceae.Corynebacterium_1,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.Corynebacteriaceae.Lawsonella,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.Mycobacteriaceae.Mycobacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.Nocardiaceae.Rhodococcus,taxa_Bacteria.Actinobacteria.Actinobacteria.Corynebacteriales.nan.nan,taxa_Bacteria.Actinobacteria.Actinobacteria.Frankiales.Geodermatophilaceae.Modestobacter,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Beutenbergiaceae.Salana,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Bogoriellaceae.Georgenia,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Brevibacteriaceae.Brevibacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Cellulomonadaceae.Cellulomonas,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Dermabacteraceae.Brachybacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Dermabacteraceae.Dermabacter,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Microbacteriaceae.Amnibacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Microbacteriaceae.Clavibacter,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Microbacteriaceae.Curtobacterium,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.Microbacteriaceae.Leucobacter,taxa_Bacteria.Actinobacteria.Actinobacteria.Micrococcales.

In [9]:
#Filtering
taxa = name_pivot.reset_index()
taxa_zerocounts=(taxa == 0).sum() # sum up times any columns have zero values
taxa_nonnancounts=(~taxa.isna()).sum() # sum up times any columns have non-NA values
taxa = taxa.loc[:, taxa_zerocounts/taxa.shape[0] <= .30] #this sum of zero values/num of total rows should less than 15% to proceed with the cohort
taxa

Name,public_client_id,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Bifidobacterium,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriaceae.Collinsella,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Adlercreutzia,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Bacteroidaceae.Bacteroides,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Marinifilaceae.Odoribacter,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Rikenellaceae.Alistipes,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Tannerellaceae.Parabacteroides,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Streptococcaceae.Streptococcus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.Christensenellaceae_R-7_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiaceae_1.Clostridium_sensu_stricto_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiales_vadinBB60_group.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_AD3011_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_UCG-001,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Agathobacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Anaerostipes,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Blautia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_3,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Dorea,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Fusicatenibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.GCA-900066575,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnoclostridium,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospira,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_FCS020_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_ND3007_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_NK4A136_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-001,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-004,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-010,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Marvinbryantia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Roseburia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Peptostreptococcaceae.Intestinibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Peptostreptococcaceae.Romboutsia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Anaerotruncus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Butyricicoccus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Candidatus_Soleaferrea,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.DTU089,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Faecalibacterium,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Flavonifractor,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.GCA-900066225,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Intestinimonas,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Negativibacillus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Oscillibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Oscillospira,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Ruminic

In [10]:
#Obtain "eGFR" calculated GFR data (credit: Alexandra Ralevski) code chunk below:
################################################################################################
chems = adi.get_snapshot('chemistries', clean=True)
cl = adi.get_snapshot('clients', clean=True)
chems_dems = join_clients(cl, chems)
#these are the relevant column names
[col for col in chems.columns if 'CREAT' in col]
[col for col in chems.columns if 'GFR' in col]
def f(sex, age, creatinine): 
    if sex == 'F':
        if (creatinine/0.7) <=1:
            return 142*(((creatinine/0.7)**-0.241) * (0.9938**age) * 1.012)
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age) * 1.012)
    if sex == 'M':
        if (creatinine/0.9) <=1:
            return 142*(((creatinine/0.9)**-0.302) * (0.9938**age))
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age))

creat_calc = chems_dems[['public_client_id','sex', 'age', 'CREATININE_ENZ__SER']]

creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x.CREATININE_ENZ__SER), axis=1)
creat_calc.to_csv('eGFR.csv',index = False)
################################################################################################

#prepare eGFR dataframe:
eGFR = creat_calc
eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR[['eGFR']] #select relevant columns
#drop duplicate entries to reduce 12316 entries across all individuals
#to 6133 individuals
eGFR = eGFR.reset_index().drop_duplicates('public_client_id')
eGFR

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,eGFR
0,01000552,NaN
1,01000978,NaN
2,01001181,NaN
3,01001298,99.024207
4,01001467,100.959522
...,...,...
12307,QIRL,111.687296
12311,ZU273983,118.179626
12312,ZU612255,70.256126
12313,ZU621944,120.472213


In [12]:
# Obtain vendor from snapshot
dash = adi.get_snapshot('microbiome_diversity')
dash = dash.sort_values('days_in_program',ascending=True) # Sort by days in program first
dash = dash.drop_duplicates(subset='public_client_id') # drop duplicate PCIs, keeping first day
dash = dash.sort_values('public_client_id',ascending=True)
dash = dash.set_index('public_client_id')
dash = dash[['vendor_dashboard']]
dash = dash[dash.notna()]
dash = dash.reset_index()
dash
#3698 individuals with vendor information

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,vendor_dashboard
0,01000261,Second Genome
1,01001298,research-microbiome
2,01001621,research-microbiome
3,01001661,research-microbiome
4,01001798,research-microbiome
...,...,...
3693,MUMA,Second Genome
3694,QIRL,Second Genome
3695,ZU273983,research-microbiome
3696,ZU621944,research-microbiome


In [13]:
#Now, get the Assessments data
bio=adi.get_snapshot('clients')
a = bio['sex']
b = bio['age']
c = bio['public_client_id']
biodf = pd.DataFrame({'sex':a, 'age':b, 'public_client_id':c})
biodf = biodf[['sex','age','public_client_id']]
#biodf = biodf.set_index('public_client_id')
biodf #6133 individuals

#Get BMI data
bmi = adi.get_snapshot('weight_monthly')
bmi = bmi[['public_client_id','days_in_program','BMI_CALC']]
bmi = bmi.sort_values('days_in_program',ascending=True)
bmi = bmi.sort_values('public_client_id',ascending=True)
bmi = bmi.drop('days_in_program', axis=1)
bmi = bmi.drop_duplicates('public_client_id')
#bmi = bmi.set_index('public_client_id')
bmi = bmi.dropna()
bmi #5567 individuals


#Merge final dataframe and output df for R analysis:
output_df = pd.merge(biodf,bmi)
output_df = pd.merge(output_df,dash)
output_df = pd.merge(output_df,bowel)
output_df = pd.merge(output_df,eGFR)
output_df = pd.merge(output_df,taxa)
output_df = output_df.set_index('public_client_id')
output_df.to_csv('gut.csv')
micro = output_df
micro

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,sex,age,BMI_CALC,vendor_dashboard,bowel,eGFR,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Bifidobacterium,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriaceae.Collinsella,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Adlercreutzia,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Bacteroidaceae.Bacteroides,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Marinifilaceae.Odoribacter,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Rikenellaceae.Alistipes,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Tannerellaceae.Parabacteroides,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Streptococcaceae.Streptococcus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.Christensenellaceae_R-7_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiaceae_1.Clostridium_sensu_stricto_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiales_vadinBB60_group.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_AD3011_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_UCG-001,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Agathobacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Anaerostipes,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Blautia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_3,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Dorea,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Fusicatenibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.GCA-900066575,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnoclostridium,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospira,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_FCS020_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_ND3007_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_NK4A136_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-001,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-004,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Lachnospiraceae_UCG-010,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Marvinbryantia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Roseburia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Peptostreptococcaceae.Intestinibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Peptostreptococcaceae.Romboutsia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Anaerotruncus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Butyricicoccus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Candidatus_Soleaferrea,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.DTU089,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Faecalibacterium,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Flavonifractor,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.GCA-900066225,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Intestinimonas,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Negativibacillus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Oscillibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Ruminococcaceae.Oscillospira,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales

In [14]:
#replace F with W:
micro = micro.replace(to_replace = 'F', value = 'W')

# Taxonomy metadata
# Algorithm provided by Christian Diener, PhD:
###################################################
taxa_table = micro.iloc[:,6:].reset_index()
taxa_table = taxa.iloc[:,1:].columns  # to get only the taxa columns
taxa_table = taxa_table[taxa_table.str.startswith("taxa_")]
taxa_table = taxa_table.str.replace("taxa_", "")  # to remove the leading `genus_`
taxa_table = taxa_table.to_series().str.split(".", expand=True)# will split on the dot and save it into the columns of a new DataFrame
taxa_table.columns = ["kingdom", "phylum", "class", "order", "family", "genus"
                     ]# set the column names
taxa_table
taxa_table.to_csv("taxa.csv", sep="\t")
####################################################

taxa = taxa.dropna()
micro = micro.dropna()

# Taxa count data:
micro.iloc[:,6:].reset_index().to_csv('gut.csv',index = False)
# Heat map data:
micro.iloc[:,6:].reset_index().to_csv('gut_hm.csv')
# Full data:
micro.reset_index().to_csv('gut_full.csv', index = False)